In [91]:
import pandas as pd
from bs4 import BeautifulSoup
import json
import re


In [89]:
path = "/Users/ahmedzaiou/Documents/ProjetsApps/TrackPharmic/Datascraping/dictionnaire_new-0-200.json"

def digimonLoad(petName):
    dStat = {}
    with open(petName) as f:
        pet = json.loads(f.read())
    return pet
    
data = digimonLoad(path)

In [90]:
def extract_med_info(match):
     
    soup = BeautifulSoup(match, 'html.parser')
    # Créer un dictionnaire pour les données
    data = soup.text.split('\n') 
    button_data = {
        "Nom": data[1].split(':')[0].strip(), 
        "caracteristique": data[1].split(':')[1].strip(), 
        "Code EAN-13" : data[3].split(':')[1].strip(),
        "Médicament GENERIQUE" : data[5].split(' ')[2].strip()
    }
    return button_data
def extract_prices(match):
    match= match.replace("Substance active (DCI) &amp; dosage", "Substance dosage") 
    soup = BeautifulSoup(match, 'html.parser')

    # Extraire les informations et structurer en JSON
    price_officine = soup.find('div', class_='text-danger').get_text(strip=True)
    price_public = soup.find(string="Prix Public de Vente").find_next('div').get_text(strip=True)
    price_remboursement = soup.find(string="Prix base remboursement").find_next('div').get_text(strip=True)
    price_hospitalier = soup.find(string="Prix Hospitalier").find_next('div').get_text(strip=True)
    substance_active = soup.find(string="Substance dosage ").find_next('div').get_text(strip=True)
    classe_therapeutique = soup.find(string="Classe Thérapeutique").find_next('div').get_text(strip=True)

    # Créer un dictionnaire pour les données
    div_data = {
        "Prix Officine": price_officine,
        "Prix Public de Vente": price_public,
        "Prix base remboursement": price_remboursement,
        "Prix Hospitalier": price_hospitalier,
        "Substance active (DCI) & dosage": substance_active,
        "Classe Thérapeutique": classe_therapeutique
    }
    return div_data

In [105]:
def get_matches(name):
    pattern_button = r'<button[^>]*class="btn btn-danger btn-block mt-1 text-left"[^>]*>.*?</button>' 
    # Extraction des balises correspondantes
    matches_b = re.findall(pattern_button, data['CHEMOSET'][1], re.DOTALL) 
    matches_d = []
    for i in range(len(matches_b)-1):
        index_1 = data['CHEMOSET'][1].find(matches_b[i])+len(matches_b[i])
        index_2 = data['CHEMOSET'][1].find(matches_b[i+1])
        matches_d.append(data['CHEMOSET'][1][index_1:index_2])
    index_1 = data['CHEMOSET'][1].find(matches_b[-1])+len(matches_b[-1])
    index_2 = data['CHEMOSET'][1].find('</div></div><div _ngcontent-tkn-c49="" class="col-xl-6 col-lg-6">')
    matches_d.append(data['CHEMOSET'][1][index_1:index_2])
    return matches_b, matches_d

In [109]:
medicaments = []
for i in data: 
    matches_b, matches_d = get_matches(i)
    for match_b,match_d in zip(matches_b, matches_d):
        b_json = extract_med_info(match_b)
        d_json = extract_prices(match_d) 
        b_json.update(d_json)
        medicaments.append(b_json)



In [113]:
df = pd.DataFrame(medicaments)
    

In [117]:
df['Prix Public de Vente'] = df['Prix Public de Vente'].str.replace(' DH', '').astype(float)
df['Prix base remboursement'] = df['Prix base remboursement'].str.replace(' DH', '').astype(float)
df['Prix Hospitalier'] = df['Prix Hospitalier'].str.replace(' DH', '').astype(float)

In [119]:
df.to_csv("/Users/ahmedzaiou/Documents/ProjetsApps/TrackPharmic/Datascraping/scrapedData/medicament-0-200.")

In [122]:
df.columns

Index(['Nom', 'caracteristique', 'Code EAN-13', 'Médicament GENERIQUE',
       'Prix Officine', 'Prix Public de Vente', 'Prix base remboursement',
       'Prix Hospitalier', 'Substance active (DCI) & dosage',
       'Classe Thérapeutique'],
      dtype='object')